In [1]:
### Data Source ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression

tickers = ['3M', 'BA', 'BAC', 'FB', 'GE', 'GOOG', 'GS', 'JPM', 'TWTR']
stock_data = pd.DataFrame()
for ticker in tickers:
    data = pd.read_csv('data/' + ticker + '.csv')
    stock_data[ticker] = data['Adj Close']

In [2]:
stock_data.index = data['Date']
stock_data.head()

,3M,BA,BAC,FB,GE,GOOG,GS,JPM,TWTR
Date,,,,,,,,,
2018-01-02,220.075348,282.886383,28.525099,181.419998,16.709501,1065.000000,245.600204,101.684258,24.510000
2018-01-03,220.065979,283.801239,28.429695,184.669998,16.867491,1082.479980,243.313950,101.787872,24.450001
2018-01-04,222.942566,282.724396,28.801765,184.330002,17.220642,1086.400024,246.714539,103.246048,23.990000
2018-01-05,224.679718,294.322296,28.935326,186.850006,17.229935,1102.229980,245.456116,102.583244,24.320000
2018-01-08,223.951233,295.570740,28.734983,188.279999,16.988306,1106.939941,241.892227,102.734734,24.590000


In [6]:
# FF3 factor data
factor_data = pd.read_csv('data/F-F_Research_Data_Factors_daily.csv', skiprows=4)
factor_data.dropna(inplace=True)
factor_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
factor_data['Date'] = pd.to_datetime(factor_data['Date'].values, format='%Y%m%d')
factor_data['Date'] = factor_data['Date'].dt.date
factor_data.set_index(factor_data['Date'].apply(lambda x: x.strftime('%Y-%m-%d')), inplace=True)
del factor_data['Date']

factor_data.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.24,-0.28,0.009
1926-07-02,0.45,-0.32,-0.08,0.009
1926-07-06,0.17,0.27,-0.35,0.009
1926-07-07,0.09,-0.59,0.03,0.009
1926-07-08,0.21,-0.36,0.15,0.009


In [4]:
all_data = pd.concat([stock_data.pct_change(), factor_data], axis=1, join='outer', sort=False)
all_data.dropna(inplace=True)
all_data.head()

,3M,BA,BAC,FB,GE,GOOG,GS,JPM,TWTR,Mkt-RF,SMB,HML,RF
2018-01-03,-0.000043,0.003234,-0.003345,0.017914,0.009455,0.016413,-0.009309,0.001019,-0.002448,0.59,-0.39,-0.21,0.005
2018-01-04,0.013071,-0.003794,0.013087,-0.001841,0.020937,0.003621,0.013976,0.014326,-0.018814,0.42,-0.26,0.24,0.005
2018-01-05,0.007792,0.041022,0.004637,0.013671,0.000540,0.014571,-0.005101,-0.006420,0.013756,0.66,-0.34,-0.27,0.005
2018-01-08,-0.003242,0.004242,-0.006924,0.007653,-0.014024,0.004273,-0.014519,0.001477,0.011102,0.19,-0.16,0.07,0.005
2018-01-09,0.006214,0.026697,0.004980,-0.002178,0.015317,-0.000614,0.008459,0.005069,-0.017080,0.15,-0.36,-0.02,0.005


In [5]:
linear_regressor = LinearRegression()  # create object for the class
for ticker in tickers:
    X = all_data[['Mkt-RF', 'SMB', 'HML']].values
    Y = all_data[ticker].values
    reg = linear_regressor.fit(X, Y)  # perform linear regression
    print('R-square for ' + ticker + ': %.4f' % reg.score(X, Y))

R-square for 3M: 0.5020
R-square for BA: 0.3497
R-square for BAC: 0.7167
R-square for FB: 0.3730
R-square for GE: 0.1879
R-square for GOOG: 0.6211
R-square for GS: 0.6317
R-square for JPM: 0.7653
R-square for TWTR: 0.2705
